Reference : https://colab.research.google.com/drive/17NbbGViskUY6G-iS5HdB7h8yI7xrRrIY?source=post_page-----d72659fc83c9--------------------------------#scrollTo=3QVfMmAqSyGd

Fine tune cost = (base training cost per 1M input tokens ÷ 1M) × number of tokens in the input file × number of epochs trained


For a training file with 100,000 tokens trained over 3 epochs, the expected cost would be:


Around 0.90 USD with gpt-4o-mini-2024-07-18 after the free period ends on September 23, 2024.


Around 2.40 USD with gpt-3.5-turbo-0125.

You can view a loss graph at https://platform.openai.com/finetune/ in your OpenAI account.

In [ ]:
model = ["gpt-4o-2024-08-06",#โมเดลที่รองรับการ Fine tune
"gpt-4o-mini-2024-07-18",
"gpt-4-0613",
"gpt-3.5-turbo-0125",
"gpt-3.5-turbo-1106",
"gpt-3.5-turbo-0613",
"babbage-002",
"davinci-002"]

GPT-4o mini FINE TUNING


In [ ]:
import shutil, os, subprocess
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install --upgrade openai jsonlines

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
import json
with open('myconfig.json') as data_file:
  myconfig = json.load(data_file)
print(myconfig.keys())

dict_keys(['OPEN_API_KEY', 'OPENAI_API_KEY', 'DB_HOST', 'DB_PORT', 'DB_USER', 'DB_PASSWORD', 'HF_TOKEN', 'CLIPDROP_KEY', 'MJ_APIKEY_PUBLIC', 'MJ_APIKEY_PRIVATE', 'MYEMAIL'])


NOW START

In [ ]:
import os
import sys
import openai


#use.env file
#openai.organization = os.environ['ORGANIZATION']
#openai.api_key  = os.environ['OPENAI_API_KEY']
os.environ["OPENAI_API_KEY"] = input("Paste your OpenAI key here and hit enter:")

In [ ]:
from openai import OpenAI
import jsonlines
client = OpenAI()

Result before fine tune

In [ ]:
import json
completion = client.chat.completions.create(
  model="gpt-4o-mini-2024-07-18",
  messages=[
    {"role": "system", "content": """Given a merger acquisition news headline, provide the following fields in a JSON dict, where 'applicable: "acquirer" and "deal_size"."""},
    {"role": "user", "content": "TechGiant Inc. announced the acquisition of a $15 billion stake in Innovatech Solutions!"}
  ]
)
print(completion.choices[0].message.content)
#pprint(json.loads(completion.choices[0].message.content))

```json
{
  "acquirer": "TechGiant Inc.",
  "deal_size": "$15 billion"
}
```


In [ ]:
completion = client.chat.completions.create(
  model="gpt-4o-mini-2024-07-18",
  messages=[
    {"role": "system", "content": """Given a merger acquisition news headline, provide the following fields in a JSON dict, where 'applicable: "acquirer" and "deal_size"."""},
    {"role": "user", "content": "Leading Edge Capital has taken over Sunrise Enterprises in a deal valued at $250 million."}
  ])
print(completion.choices[0].message.content)

```json
{
  "acquirer": "Leading Edge Capital",
  "deal_size": "$250 million"
}
```


In [ ]:
%%time
items =[
{"messages": [{"role": "system", "content": "Given a merger acquistion news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "Listed media and gaming firm Nazara Technologies said on Friday that it acquired the remaining 48.42% stake it did not hold in Paper Boat Apps for Rs 300 crore, making it a wholly owned subsidiary."}, {"role": "assistant", "content": "{\"acquirer\": \" Nazara Technologies\", \"deal_size\": \"Rs 300 crore\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquistion news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "Venturi Partners (Venturi), a prominent growth equity firm specialising in consumer investments across India and Southeast Asia, announced the purchase of a $27 million stake in K12 Techno Services"}, {"role": "assistant", "content": "{\"acquirer\": Venturi Partners (Venturi)\", \"deal_size\": \"$27 million\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquisition news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "TechGiant Inc. announced the acquisition of a $15 billion stake in Innovatech Solutions."}, {"role": "assistant", "content": "{\"acquirer\": \"TechGiant Inc.\", \"deal_size\": \"$15 billion\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquisition news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "Future Ventures has completed its $3.5 million purchase of GreenTech Innovations."}, {"role": "assistant", "content": "{\"acquirer\": \"Future Ventures\", \"deal_size\": \"$3.5 million\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquisition news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "Leading Edge Capital has taken over Sunrise Enterprises in a deal valued at $250 million."}, {"role": "assistant", "content": "{\"acquirer\": \"Leading Edge Capital\", \"deal_size\": \"$250 million\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquistion news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "Listed media and gaming firm Nazara Technologies said on Friday that it acquired the remaining 48.42% stake it did not hold in Paper Boat Apps for Rs 300 crore, making it a wholly owned subsidiary."}, {"role": "assistant", "content": "{\"acquirer\": \" Nazara Technologies\", \"deal_size\": \"Rs 300 crore\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquistion news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "Venturi Partners (Venturi), a prominent growth equity firm specialising in consumer investments across India and Southeast Asia, announced the purchase of a $27 million stake in K12 Techno Services"}, {"role": "assistant", "content": "{\"acquirer\": Venturi Partners (Venturi)\", \"deal_size\": \"$27 million\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquisition news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "TechGiant Inc. announced the acquisition of a $15 billion stake in Innovatech Solutions."}, {"role": "assistant", "content": "{\"acquirer\": \"TechGiant Inc.\", \"deal_size\": \"$15 billion\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquisition news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "Future Ventures has completed its $3.5 million purchase of GreenTech Innovations."}, {"role": "assistant", "content": "{\"acquirer\": \"Future Ventures\", \"deal_size\": \"$3.5 million\" }"}]},
{"messages": [{"role": "system", "content": "Given a merger acquisition news headline, provide the following fields in a JSON dict, where applicable: \"acquirer\"  and \"deal_size\"."}, {"role": "user", "content": "Leading Edge Capital has taken over Sunrise Enterprises in a deal valued at $250 million."}, {"role": "assistant", "content": "{\"acquirer\": \"Leading Edge Capital\", \"deal_size\": \"$250 million\" }"}]},

]

with jsonlines.open('mna.jsonl', 'w') as writer:
    writer.write_all(items)

CPU times: user 93 µs, sys: 777 µs, total: 870 µs
Wall time: 746 µs


In [ ]:
from pprint import pprint
with jsonlines.open("mna.jsonl") as file:
    data = list(file.iter())

pprint(data)

[{'messages': [{'content': 'Given a merger acquistion news headline, provide '
                           'the following fields in a JSON dict, where '
                           'applicable: "acquirer"  and "deal_size".',
                'role': 'system'},
               {'content': 'Listed media and gaming firm Nazara Technologies '
                           'said on Friday that it acquired the remaining '
                           '48.42% stake it did not hold in Paper Boat Apps '
                           'for Rs 300 crore, making it a wholly owned '
                           'subsidiary.',
                'role': 'user'},
               {'content': '{"acquirer": " Nazara Technologies", "deal_size": '
                           '"Rs 300 crore" }',
                'role': 'assistant'}]},
 {'messages': [{'content': 'Given a merger acquistion news headline, provide '
                           'the following fields in a JSON dict, where '
                           'applicable: "a

Upload Training File

In [ ]:
result_file = client.files.create(
  file=open("mna.jsonl", "rb"),
  purpose="fine-tune"
)

pprint(result_file)

FileObject(id='file-svZbE9osJznl9ZhDQhcHr43x', bytes=4602, created_at=1725086439, filename='mna.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)


In [ ]:
param_training_file_name = result_file.id
pprint(param_training_file_name)

'file-svZbE9osJznl9ZhDQhcHr43x'


Create Fine Tuned Model

In [ ]:
result_job = client.fine_tuning.jobs.create(
  training_file=param_training_file_name,
  model="gpt-4o-mini-2024-07-18",
  hyperparameters={
    "n_epochs":1   ##ปรับ hyperparameter ได้เเค่ epoch (default = 10), learning rate (default = 1.8), batch size (default = 1)
  }
)

pprint(result_job)

FineTuningJob(id='ftjob-msedUo8ByMCpe8mmkx4RvdyQ', created_at=1725087089, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=1, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-iuIMWXlUahOdA3Lvv9cIzHKU', result_files=[], seed=621682584, status='validating_files', trained_tokens=None, training_file='file-svZbE9osJznl9ZhDQhcHr43x', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)


In [ ]:
param_file_tune_job_id = result_job.id
pprint(param_file_tune_job_id)

'ftjob-msedUo8ByMCpe8mmkx4RvdyQ'


Retrieve the state of a fine-tune

In [ ]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(param_file_tune_job_id)

FineTuningJob(id='ftjob-msedUo8ByMCpe8mmkx4RvdyQ', created_at=1725087089, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=1, batch_size=1, learning_rate_multiplier=1.8), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-iuIMWXlUahOdA3Lvv9cIzHKU', result_files=[], seed=621682584, status='running', trained_tokens=None, training_file='file-svZbE9osJznl9ZhDQhcHr43x', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(param_file_tune_job_id).status

'running'

In [ ]:
# Retrieve the state of a fine-tune
from time import sleep
while client.fine_tuning.jobs.retrieve(param_file_tune_job_id).status != 'succeeded':
  sleep(10)


In [ ]:
# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(param_file_tune_job_id).status

'succeeded'

In [ ]:
param_file_tune_model = client.fine_tuning.jobs.retrieve(param_file_tune_job_id).fine_tuned_model
pprint(param_file_tune_model)

'ft:gpt-4o-mini-2024-07-18:personal::A2CHuGvg'


Result after fine tune

In [ ]:
completion = client.chat.completions.create(
  model=param_file_tune_model,
  messages=[
    {"role": "system", "content": """Given a merger acquisition news headline, provide the following fields in a JSON dict, where 'applicable: "acquirer" and "deal_size"."""},
    {"role": "user", "content": "TechGiant Inc. announced the acquisition of a $15 billion stake in Innovatech Solutions!"}
  ]
)
pprint(json.loads(completion.choices[0].message.content))

{'applicable': {'acquirer': 'TechGiant Inc.', 'deal_size': '$15 billion'}}


In [ ]:
completion = client.chat.completions.create(
  model=param_file_tune_model,
  messages=[
    {"role": "system", "content": """Given a merger acquisition news headline, provide the following fields in a JSON dict, where 'applicable: "acquirer" and "deal_size"."""},
    {"role": "user", "content": "Leading Edge Capital has taken over Sunrise Enterprises in a deal valued at $250 million."}
  ]
)
pprint(json.loads(completion.choices[0].message.content))

{'applicable': {'acquirer': 'Leading Edge Capital',
                'deal_size': '$250 million'}}
